Careless responder feature engineering and analysis (inital focus on psych_flex survey only)

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt
from numpy import median

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/data_final_combined.pkl'

In [ ]:
#read in csv containing data from all surveys
data = pd.read_pickle(data_path)
data = pd.DataFrame(data = data)
#data_PF.reset_index(inplace=True)

print('Original data shape:\n', data.shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', data['ParticipantID'].unique().shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', data['MitreID'].unique().shape, '\n')
#how much missing data is there?
print('Original data missing value counts:\n', data.isnull().sum(), '\n')
#what is the data type of each column?
print('Original data data types:\n', data.info(), '\n')

In [ ]:
data['survey_type'].value_counts()

In [ ]:
#should be 71
len(data['wave_study_day'].unique())

In [ ]:
data.head()

### notes for CR features for Psych Flex

Should have answered every question

Longstring
- Legitimate longstrings of  ≥ 8 are unlikely for response “5”
    - make column with longest string
    - make column with number that longest string consisted of

Semantic consistency
- Legitimate scores of pf_mgt=5 are almost impossible

Semantic antonyms
- Not applicable

Semantic synonyms 
- not applicable 


In [ ]:
#split off completed PF and related columns
psych_flex_only = data[(data['survey_type'] == 'psych_flex') & (data['completed'] == 1.0)]
psych_flex_only = psych_flex_only[['MitreID', 'ParticipantID', 'PrimaryUnit', 'SmartPhone',
       'Sex', 'Shift', 'Wave', 'survey_id', 'survey_type', 
       'date', 'time','start_delay', 
       'time_to_complete', 'activity_num', 'location_num',
       'activity', 'pf_03', 'pf_04', 'pf_05', 'pf_06',
       'pf_07', 'pf_08', 'pf_09', 'pf_10', 'pf_11', 'pf_12', 'pf_13',
       'pf_14', 'pf_15', 'pf_mgt', 'exp_0', 'exp_1', 'exp_2', 'exp_3',
       'exp_4', 'exp_5', 'exp_6', 'exp_7', 'exp_8', 'exp_9', 'exp_10',
       'exp_11', 'exp_12', 'exp_13', 'exp_neg', 'exp_pos', 'exp_neut',
       'wave_study_date', 'wave_study_day',
       'survey_count', 'day_of_week', 'working', 'at_work']]

psych_flex_only.shape
psych_flex_only['ParticipantID'].unique().shape

In [ ]:
path_demog_prepost = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/Demog, PRE, PST survey composites.csv'

In [ ]:
#read in data_demog_prepost, contains demographic information and pre/post questionaires
data_demog_prepost = pd.read_csv(path_demog_prepost)
data_demog_prepost = pd.DataFrame(data = data_demog_prepost)
print('Original demog_prepost shape:\n', data_demog_prepost.shape, '\n')
#replace blnaks with nans
data_demog_prepost = data_demog_prepost.replace(' ', np.nan)
#ensure no replicate ID (eg one row per participant in study)
print('Original demog_prepost unique IDs:\n', data_demog_prepost['ID'].unique().shape, '\n')
print('Original demog_prepost missing value couts:\n', data_demog_prepost.isnull().sum(), '\n')
print('Original demog_prepost data types:\n', data_demog_prepost.info(), '\n')
data_demog_prepost.head()

In [ ]:
#add the information contained in data_demog_prepost to psych_flex_CR
#first create new data table of data_demog_prepost that contains the same number of rows for each participant in that is in psych_flex_CR
#(e.g. replicate data_demog_prepost so same length as psych_flex_CR for each participant)

participants = psych_flex_only['MitreID'].unique()

psych_flex_demog = pd.DataFrame()

for part in participants:
    df_part_long = pd.concat([data_demog_prepost[data_demog_prepost['ID'] == part]]*len(psych_flex_only[psych_flex_only['MitreID'] ==  part]), ignore_index=True)
    df_part_long.reset_index(inplace=True)
    psych_flex_demog_int = pd.concat([df_part_long, psych_flex_only[psych_flex_only['MitreID'] ==  part].reset_index()], axis = 1)
    psych_flex_demog = psych_flex_demog.append(psych_flex_demog_int)

#confirm the two data tables are now the same lenght
print('psych_flex_only and psych_flex_demog are the same length:', psych_flex_only.shape[0] == psych_flex_demog.shape[0])
print(psych_flex_only.shape[0])
print(psych_flex_demog.shape[0])
print('does the math make sense?', psych_flex_demog.shape[0] == (psych_flex_demog['ID'].values == psych_flex_demog['MitreID'].values).sum())
psych_flex_demog.head()

In [ ]:
#split off final variables of interest
psych_flex_demog_features = ['MitreID', 'ParticipantID', 'PrimaryUnit', 'SmartPhone',
       'Sex', 'Shift', 'bornUS', 'lang', 'englyrs', 'Wave', 'survey_id', 'survey_type',
       'date', 'time',
        'wave_study_date', 'wave_study_day',
       'survey_count', 'day_of_week', 'working', 'at_work', 'start_delay',
       'time_to_complete', 'activity_num', 'location_num',
       'activity', 'pf_03', 'pf_04', 'pf_05', 'pf_06',
       'pf_07', 'pf_08', 'pf_09', 'pf_10', 'pf_11', 'pf_12', 'pf_13',
       'pf_14', 'pf_15', 'pf_mgt', 'exp_0', 'exp_1', 'exp_2', 'exp_3',
       'exp_4', 'exp_5', 'exp_6', 'exp_7', 'exp_8', 'exp_9', 'exp_10',
       'exp_11', 'exp_12', 'exp_13', 'exp_neg', 'exp_pos', 'exp_neut', 
        'ocb', 'inter.deviance', 'org.deviance', 'extraversion',
       'agreeableness', 'conscientiousness', 'neuroticism', 'openness',
       'psqi', 'fatigue', 'Stress', 
       'Flexibility', 'Inflexibility', 'Engagement']
        
PF_demog_df = psych_flex_demog[psych_flex_demog_features]
PF_demog_df.reset_index(inplace=True)
PF_demog_df.head()

In [ ]:
#create study date bins
PF_demog_df['wave_study_date_bin'] = pd.cut(PF_demog_df['wave_study_day'], 5)
PF_demog_df['wave_study_date_bin'].value_counts()

## Survey level features and clustering

In [ ]:
#long string analysis (e.g. max length of same number answered for pf_03:pf_15)
#create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

max_strings = []
max_answers = []

for index, row in PF_demog_df.iterrows():
    
    groups = groupby(row['pf_03':'pf_15'])
    result = [(label, sum(1 for _ in group)) for label, group in groups]

    max_pair = max(result, key=lambda x:x[1])
    max_string_length = max_pair[1]
    max_answer = max_pair[0]

    max_strings.append(max_string_length)
    
    max_answers.append(max_answer)

PF_demog_df['longest_string_count'] = max_strings
PF_demog_df['longest_string_answer'] = max_answers

In [ ]:
#create feature that is surevy response skew
PF_demog_df['skew'] = PF_demog_df.loc[:, 'activity':'pf_mgt'].skew(axis=1)

In [ ]:
PF_demog_df.dropna(subset=['skew'], inplace=True)

In [ ]:
PF_survey_features = PF_demog_df[['MitreID', 'longest_string_count', 'skew', 'time_to_complete']]
PF_survey_features.set_index('MitreID', inplace=True)
PF_survey_features.head()

In [ ]:
sns.pairplot(PF_survey_features)

In [ ]:
# center and scale the data
scaler = StandardScaler()

PF_survey_features_scaled = scaler.fit_transform(PF_survey_features)

In [ ]:
k_range = range(2,10)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(PF_survey_features_scaled)
    scores.append(silhouette_score(PF_survey_features_scaled, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.title('PF kmeans at survey level')

In [ ]:
PF_km_survey = KMeans(n_clusters=6,random_state=1234)
PF_km_survey.fit(PF_survey_features_scaled)
PF_survey_features['kmeans_scaled_survey'] = ["cluster_" + str(label) for label in PF_km_survey.labels_ ]
PF_survey_features.head()

In [ ]:
PF_survey_features['kmeans_scaled_survey'].value_counts()

In [ ]:
sns.pairplot(PF_survey_features, hue = 'kmeans_scaled_survey')

In [ ]:
#add cluster column to PF survey df
participants = PF_demog_df['MitreID'].unique()

for part in participants:
    PF_demog_df.loc[PF_demog_df['MitreID'] == part, 'kmeans_cluster_survey'] = PF_survey_features[PF_survey_features.index == part]['kmeans_scaled_survey'].values
    
PF_demog_df.head()

## Participant level features and clustering

In [ ]:
#create participant level CR features df
PF_part_features = pd.DataFrame()
PF_part_features['MitreID'] = PF_demog_df['MitreID'].unique()

#long string count features
PF_part_features['ls_count_ave'] = PF_demog_df.groupby('MitreID')['longest_string_count'].mean().values
#PF_CRpart_features['ls_count_min'] = PF_demog_df.groupby('MitreID')['longest_string_count'].min().values
#PF_CRpart_features['ls_count_max'] = PF_demog_df.groupby('MitreID')['longest_string_count'].max().values
PF_part_features['ls_count_skew'] = PF_demog_df.groupby('MitreID')['longest_string_count'].skew().values

#long string answer choice features
#PF_CRpart_features['ls_num_ave'] = PF_demog_df.groupby('MitreID')['longest_string_answer'].mean().values
#PF_CRpart_features['ls_num_min'] = PF_demog_df.groupby('MitreID')['longest_string_answer'].min().values
#PF_CRpart_features['ls_num_max'] = PF_demog_df.groupby('MitreID')['longest_string_answer'].max().values
#PF_CRpart_features['ls_num_skew'] = PF_demog_df.groupby('MitreID')['longest_string_answer'].skew().values

#time to complete survey features
PF_part_features['ttc_ave'] = PF_demog_df.groupby('MitreID')['time_to_complete'].mean().values
#PF_CRpart_features['ttc_min'] = PF_demog_df.groupby('MitreID')['time_to_complete'].min().values
#PF_CRpart_features['ttc_max'] = PF_demog_df.groupby('MitreID')['time_to_complete'].max().values
PF_part_features['ttc_skew'] = PF_demog_df.groupby('MitreID')['time_to_complete'].skew().values

#surveys completed features
#PF_CRpart_features['survey_ratio'] = PF_demog_df.groupby('MitreID')['survey_count'].count().values / 50
#PF_CRpart_features['survey_max'] = PF_demog_df.groupby('MitreID')['survey_count'].max().values

PF_part_features.set_index('MitreID', inplace=True)
PF_part_features.dropna(subset=['ls_count_skew'], inplace=True)
PF_part_features.head()

In [ ]:
sns.pairplot(PF_part_features)

In [ ]:
# center and scale the data
scaler = StandardScaler()

PF_part_features_scaled = scaler.fit_transform(PF_part_features)

In [ ]:
k_range = range(2,10)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(PF_part_features_scaled)
    scores.append(silhouette_score(PF_part_features_scaled, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.title('PF kmeans at participant level')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')

In [ ]:
PF_km_part = KMeans(n_clusters=3,random_state=1234)
PF_km_part.fit(PF_part_features_scaled)
PF_part_features['kmeans_scaled_part'] = ["cluster_" + str(label) for label in PF_km_part.labels_ ]
PF_part_features.head()

In [ ]:
PF_part_features['kmeans_scaled_part'].value_counts()

In [ ]:
sns.pairplot(PF_part_features, hue='kmeans_scaled_part')

In [ ]:
#add cluster column to PF survey df
participants = PF_part_features.index

for part in participants:
    PF_demog_df.loc[PF_demog_df['MitreID'] == part, 'kmeans_cluster_part'] = PF_part_features[PF_part_features.index == part]['kmeans_scaled_part'].values
    
PF_demog_df.head()

In [ ]:
#add cluster ratio to PF survey df
participants = PF_demog_df['MitreID'].unique()

for part in participants:
    PF_demog_df.loc[PF_demog_df['MitreID'] == part, 'kmeans_cluster_survey_ratio'] = \
    PF_demog_df[(PF_demog_df['MitreID'] == part) & (PF_demog_df['kmeans_cluster_survey'] == 'cluster_0')].shape[0] / \
    PF_demog_df[PF_demog_df['MitreID'] == part].shape[0]
    
PF_demog_df.head()

## Participant level clustering based off clustering of surveys ratio

In [ ]:
#create participant level CR features df
PF_part_ratio_features = pd.DataFrame()
PF_part_ratio_features['MitreID'] = PF_demog_df['MitreID'].unique()

#long string count features
PF_part_ratio_features['survey_ratio_ave'] = PF_demog_df.groupby('MitreID')['kmeans_cluster_survey_ratio'].mean().values


PF_part_ratio_features.set_index('MitreID', inplace=True)
PF_part_ratio_features.dropna(subset=['survey_ratio_ave'], inplace=True)
PF_part_ratio_features.head()

In [ ]:
sns.pairplot(PF_part_ratio_features)

In [ ]:
# center and scale the data
scaler = StandardScaler()

PF_part_ratio_features_scaled = scaler.fit_transform(PF_part_ratio_features)

In [ ]:
k_range = range(2,10)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(PF_part_ratio_features_scaled)
    scores.append(silhouette_score(PF_part_ratio_features_scaled, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.title('PF kmeans at participant level based on survey clustering ratio')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')

In [ ]:
PF_km_part_ratio = KMeans(n_clusters=3,random_state=1234)
PF_km_part_ratio.fit(PF_part_ratio_features_scaled)
PF_part_ratio_features['kmeans_scaled_part_ratio'] = ["cluster_" + str(label) for label in PF_km_part_ratio.labels_ ]
PF_part_ratio_features.head()

In [ ]:
PF_part_ratio_features['kmeans_scaled_part_ratio'].value_counts()

In [ ]:
#add cluster column to PF survey df
participants = PF_part_ratio_features.index

for part in participants:
    PF_demog_df.loc[PF_demog_df['MitreID'] == part, 'kmeans_cluster_part_ratio'] = PF_part_ratio_features[PF_part_ratio_features.index == part]['kmeans_scaled_part_ratio'].values
    
PF_demog_df.head()

In [ ]:
#save to csv
PF_demog_df.to_csv('PF_clustering_3ways.csv')

#pickle to save
PF_demog_df.to_pickle('PF_clustering_3ways.pkl')

In [ ]:
g = sns.catplot(x="wave_study_date_bin", y="longest_string_count", data=PF_demog_df, kind="bar", col='kmeans_cluster_survey')
g.fig.set_size_inches(25,10)

In [ ]:
g = sns.catplot(x="wave_study_date_bin", y="longest_string_count", data=PF_demog_df, kind="bar", col='kmeans_cluster_part')
g.fig.set_size_inches(25,10)

In [ ]:
g = sns.catplot(x="wave_study_date_bin", y="kmeans_cluster_survey_ratio", data=PF_demog_df, kind="bar", col='kmeans_cluster_survey')
g.fig.set_size_inches(25,10)

In [ ]:
g = sns.catplot(x="wave_study_date_bin", y="kmeans_cluster_survey_ratio", data=PF_demog_df, kind="bar", col='kmeans_cluster_part')
g.fig.set_size_inches(25,10)

In [ ]:
PF_CR_demog_df['MitreID'].unique()

In [ ]:
unique_ids = PF_CR_demog_df['MitreID'].unique()
for u_id in unique_ids:
    g = sns.catplot(x="wave_study_day", y="longest_string_count", data=PF_CR_demog_df[PF_CR_demog_df['MitreID'] == u_id], kind="bar")
    g.fig.set_size_inches(10,5)
    #axes = g.axes
    plt.ylim([0,13])
    #g.set_ylim(0, 13)
    plt.show()

In [ ]:
#get participant ID order when sorted by longest string in the last time bin
wave_1_ls = PF_CR_demog_df[PF_CR_demog_df['wave_study_date_bin'] == PF_CR_demog_df['wave_study_date_bin'].unique()[-1]].sort_values('longest_string_count')
ls_index = wave_1_ls['MitreID'].unique()

In [ ]:
#use order from above
g = sns.catplot(x="MitreID", y="longest_string_count", data=PF_CR_demog_df, kind="bar", col='wave_study_date_bin', 
                ci=None, order=ls_index)
g.fig.set_size_inches(25,10)

In [ ]:
g = sns.FacetGrid(PF_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "ocb", "longest_string_count")
g.fig.set_size_inches(25,10)



In [ ]:
PF_CR_demog_df['Inflexibility'] = PF_CR_demog_df['Inflexibility'].astype(float)

In [ ]:
g = sns.FacetGrid(PF_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "org.deviance", "longest_string_count")
g.fig.set_size_inches(25,10)

